In [1]:
from h3 import h3
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box
from shapely.ops import transform
import pyproj
from functools import partial
import rasterio
from datetime import datetime, time
import time
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

In [2]:
def create_h3_geom_cells(extent, resolutions, table,  db_engine, export_type):
    """Create geometry for h3 cells in given extent for given resolutions levels

    Parameters:
    extentJSON (geojson): extent for covering with tiles
    resolutions([int]): array of integer h3 resolution levels
    table(string): table name for postgres database
    db_engine (sqlalchemy.engine): sqlalchemy database engine
    export_type(string): where to export 'geojson' or 'postgres'

    Returns:
    none
   """
    
    for res in resolutions:
        calc_time = time.time()
        print(f'start caclulating resolution {res}')
        set_hex = list(h3.polyfill(extent_json['features'][0]["geometry"], res=res))
        
        print(f'finish caclulating resolution {res} in {str(round(time.time() - calc_time, 2))} seconds')

        if export_type == 'postgres':
            geom_time = time.time()
            gdf = gpd.GeoDataFrame({"hex_id": set_hex})
            gdf['geometry'] = gdf["hex_id"].apply(lambda x:(Polygon(h3.h3_to_geo_boundary(x)).wkb))
            
            print(f'finish caclulating geometry fo res {res} in {str(round(time.time() - geom_time, 2))} seconds')
            
            import_time = time.time()
            gdf.to_postgis(table + str(res), db_engine, if_exists='replace')
            
            print(f'finish import to db {res} in {str(round(time.time() - import_time, 2))} seconds')

        elif export_type == 'geojson':
            proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                           pyproj.Proj('proj=isea'))
            gdf = gpd.GeoDataFrame({"hex_id": set_hex})

            gdf['geometry'] = gdf["hex_id"].apply(lambda x: Polygon(h3.h3_to_geo_boundary(x, True)))
            gdf['area'] = gdf["geometry"].apply(lambda x: transform(proj, x).area)
            gdf.to_file("{}{}.geojson".format(table, res), driver='GeoJSON')
            print('finish import to geojson {} {}'.format(res, time.asctime(time.localtime(time.time()))))

In [3]:
HOST = 'localhost'
DB = 'dggs'
USER = 'Iam'
PORT = 5432
PWD = 'postgres'

In [4]:
db_url = URL(drivername='postgresql+psycopg2', host=HOST, database=DB,
             username=USER, port=PORT, password=PWD)

In [5]:
engine = create_engine(db_url)

In [6]:
extent_JSON = gpd.GeoSeries(box(21.0, 57.0, 29.0, 60.0)).__geo_interface__

In [ ]:
create_h3_geom_cells(extent_JSON, [11], 'h3_cells', engine, 'postgres')

start caclulating resolution 11
finish caclulating resolution 11 in 866.36 seconds
